<!--BOOK_INFORMATION-->
<img style="float: right; width: 100px" src="https://raw.github.com/pyomeca/design/master/logo/logo_cropped.svg?sanitize=true">

<font size="+2">Pyosim in the cloud</font>

<font size="+1">with [pyomeca](https://github.com/pyomeca/pyom</font>a)

Romain Martinez (martinez.staps@gmail.com | [GitHub](https://github.com/romainmartinez))


<!--NAVIGATION-->
| [Contents](Index.ipynb) | [Verification](01.00-verification.ipynb) >

# Logistics

In [40]:
dot = Digraph()

dot.attr('node', shape='plaintext')
dot.node('bio', 'biological\nCôté et al. (2012)')
dot.node('kin', 'kinematics\nMartinez et al. (2019)')
dot.node('emg', 'EMG\nBouffard et al. (2019)')
dot.node('msk', 'musculoskeletal')
# dot.attr('node', shape='doublecircle')
dot.node('msd', 'MSDs')
# dot.attr('node', shape='ellipse')
dot.attr('node',  color='lightgrey')
dot.node('u', 'unobserveds')

dot.edge('bio', 'kin')
dot.edge('kin', 'emg')
dot.edge('emg', 'msk')
dot.edge('kin', 'msk')
dot.edge('msk', 'msd')
dot.edge('u', 'msd')

dot.render('msd', view=True)

'msd.pdf'

In [6]:
from graphviz import Digraph

dot = Digraph(comment='The Round Table')

dot.node('A', 'King Arthur')
dot.node('B', 'Sir Bedevere the Wise')
dot.node('L', 'Sir Lancelot the Brave')

dot.edges(['AB', 'AL'])
dot.edge('B', 'L', constraint='false')

dot.render('round-table.gv', view=True)  

'round-table.gv.pdf'

<!--NAVIGATION-->
| [Contents](Index.ipynb) | [Verification](01.00-verification.ipynb) >